## Sudoku Solver Using a Genetic Algorithm

Nikolai Lyssogor

In [1]:
from random import choice as random_choice, random, sample
import math
import copy
import numpy as np

# Data structure for sudoku board. 
# See documentation above.
class SudokuBoard:
    # Constructor: prefills is a dictionary describing the initial problem. See writeup above.
    # The argument compute_allowed_fills is used for test cases. 
    def __init__(self, prefills, compute_allowed_entries=True, verbose=False):
        # save prefills in a field.
        self.prefills = prefills
        # Initialize the dictionary for allowed entries.
        self.allowed_entries = {}
        if compute_allowed_entries:
            # Place each entry in prefills into the allowed_entries as a singleton list.
            for (coord,val) in prefills.items():
                self.allowed_entries[coord] = [val]
            # iterate through all cells
            for i in range(1, 10):
                for j in range(1, 10):
                    if (i,j) not in self.allowed_entries:
                        # Compute list of allowed entry for cell
                        self.compute_allowed_entries_for_cell( i, j)
            self.state = {}
            # Initialize self.state to a random state.
            self.initialize_state(verbose)
        else: 
            self.state = self.prefills
        
    # Method: pretty_print
    # Print the problem nicely in a human readable format.
    def pretty_print(self):
        state = self.state
        # Iterate through each row
        blk_sep = '|' + '-'*9 + '+' + '-'*9 +  '+' +  '-'*9  + '|'
        print(blk_sep)
        for row_id in range(1,10): 
            # Iterate through each column
            row_str = '|'
            for col_id in range(1,10):
                # If row is not empty
                if (row_id, col_id) in state:
                    row_str = row_str + ' '+str(state[(row_id, col_id)]) + ' '
                else:
                    row_str = row_str + '   '
                if col_id % 3 == 0:
                    row_str = row_str + '|'
            print(row_str)
            if row_id %3 == 0:
                print(blk_sep)

    # get the list of prefills for a row i
    def get_row_prefills(self, i):
        return [ self.prefills[(i, j )] for j in range(1, 10) if (i,j) in self.prefills ]

    # get the list of prefills for a column j
    def get_col_prefills(self, j):
        return [ self.prefills[(i, j )] for i in range(1, 10) if (i,j) in self.prefills ]

    # get the list of prefills for block corr. to (i,j)
    def get_blk_prefills(self, i, j):
        # first compute the block coords from (i,j)
        (a, b) = ((i-1)//3 + 1, (j-1)//3 + 1)
        # find the prefills in this block.
        return [ self.prefills[(j,k)] 
                    for j in range(a*3-2, a*3+1)
                    for k in range(b*3-2, b*3+1)
                    if (j,k) in self.prefills ]
    
    # get row i as a list
    def get_row_state(self, i):
        return [ self.state[(i, j )] for j in range(1, 10) if (i,j) in self.state ]

    # get column i as a list
    def get_col_state(self, j):
        return [ self.state[(i, j )] for i in range(1, 10) if (i,j) in self.state ]

    # get block of (i, j) as a list
    def get_blk_state(self, i, j):
        # first compute the block coords from (i,j)
        (a, b) = ((i-1)//3 + 1, (j-1)//3 + 1)
        # find the prefills in this block.
        return [ self.state[(j,k)] 
                    for j in range(a*3-2, a*3+1)
                    for k in range(b*3-2, b*3+1)
                    if (j,k) in self.state ]
    
    # same as above but input is block index not board index
    def get_blk_state_by_blk_idx(self, a, b):
        return [ self.state[(j,k)] 
                    for j in range(a*3-2, a*3+1)
                    for k in range(b*3-2, b*3+1)
                    if (j,k) in self.state ]

    # Compute the list of allowed entries for cell (i,j) that do not already
    # repeat a prefill for the row i, col j or blk corr. to (i,j)
    def compute_allowed_entries_for_cell(self, i, j):
        assert ( 1 <= i <= 9)
        assert ( 1 <= j <= 9)
        # Get the prefills in row i, column j and blk corr. to (i,j)
        lst = self.get_row_prefills(i) + self.get_col_prefills(j) + self.get_blk_prefills(i, j)
        # Get the numbers from 1 to 9 that are NOT in the list of prefills computed above.
        allowed_lst = [ k for k in range(1, 10) if k not in lst]
        # Set the dictionary entry in self.allowed_entries
        self.allowed_entries[(i,j)] = allowed_lst
        # make sure that allowed_lst is not empty. This would mean a bad problem with no solutions.
        assert len(allowed_lst) >= 1, 'Conflict detected in cell (%d, %d) -- check your sudoku problem please' %(i,j)
        return 


    # Initialize the state of the board to a random state.
    def initialize_state(self, verbose=False):
        self.state = {}
        # Prefill the items 
        for (coord, k) in self.prefills.items():
            self.state[coord] = k
        # randomly choose from the allowed entries
        for i in range(1, 10):
            for j in range(1, 10):
                if (i, j) not in self.state:
                    assert (i,j) in self.allowed_entries
                    lst = self.allowed_entries[(i,j)]
                    k  = random_choice(lst)
                    if verbose:
                        print('board.state[(%d,%d)] = %d' % (i,j,k))
                    self.state[(i,j)] = k
        return 
    
    # TODO: Implement this function
    # Compute a fitness score for this board with self.state representing the board state.
    # Important: 
    #   1. Fitness score must be <= 0
    #   2. Fitness score = 0 means problem is successfully solved.
    #   3. Lower fitness score means solution violates Sudoku property for more rows, columns and blocks.
    # You should use the description of fitness provided as part of the problem writeup.
    # You can use verbose flag to print messages by checking if verbose is True.
    # This is enabled during test cases below.
    def compute_fitness_score(self, verbose=False):
        
        # YOUR CODE HERE
        fitness_score = sum([len(set(self.get_row_state(i))) - 9 for i in range(1,10)]) + \
                        sum([len(set(self.get_col_state(i))) - 9 for i in range(1,10)]) + \
                        sum([len(set(self.get_blk_state(i,j))) - 9 for i in range(1, 10, 3) \
                                                                      for j in range(1, 10, 3)])
        return fitness_score
        
        

    #Function: make_copy
    # make a shallow copy of the all fields but a deep copy of the state so that it can be modified
    # modify the state to set entry (i,j) to k
    def make_copy(self, i, j, k):
        copy_state = copy.copy(self)
        copy_state.state= copy.deepcopy(self.state) # deep copy
        assert k in self.allowed_entries[(i,j)]
        assert self.state[(i,j)] != k
        copy_state.state[(i,j)] = k
        return copy_state
    
    #Function: get_all_neighbors
    # Get all sudoku states that differ from current state in one position.
    # Note that all cells have to be filled with some entry from self.allowed_entries
    def get_all_neighbors(self):
        lst = [ self.make_copy(i, j, l) 
               for i in range(1,10)
                   for j in range(1,10)
                       for l in self.allowed_entries[(i,j)] 
                           if (l != self.state[(i,j)]) ]
#         lst_with_fitness= [( neighbor.compute_fitness_score(), neighbor) for neighbor in lst ]
        return lst

    # Function: make_copy_new_row
    # make a shallow copy of the all fields but a deep copy of the state so that it can be modified
    # modify the state to set rows in 'rows' to the values of new_entries
    def create_child(self, rows, new_entries):
        copy_state = copy.copy(self)
        copy_state.state = copy.deepcopy(self.state)
        
        # change the rows
        for i in range(len(rows)):
            for j in range(1, 10):
                copy_state.state[(rows[i], j)] = new_entries[i][j-1]
                
        return copy_state
    
    # Similar to above function but it changes rows and columns
    def create_child_v2(self, row_nums, col_nums, blk_indices, new_cols, new_rows, new_blks):
        copy_state = copy.copy(self)
        copy_state.state = copy.deepcopy(self.state)
        
        # first: change the rows
        for i in range(len(row_nums)):
            for j in range(1, 10):
                copy_state.state[(row_nums[i], j)] = new_rows[i][j-1]
                
        # second: change the columns
        for j in range(len(col_nums)): # for each column index
            for i in range(1, 10): # for each column of the board
                copy_state.state[(i, col_nums[j])] = new_cols[j][i-1]
                
        # third: change the blocks
        for blk in range(len(blk_indices)):
            idx = 0
            (a, b) = (blk_indices[blk][0], blk_indices[blk][1])
            for j in range(a*3-2, a*3+1):
                    for k in range(b*3-2, b*3+1):
                        copy_state.state[(j,k)] = new_blks[blk][idx]
                        idx += 1
                
        # fourth: mutate random block according to a probability
        if(random() < 0.2):
            (a, b) = (sample(range(1,3),1)[0], sample(range(1,3),1)[0]) # compute random block coordinates

            for j in range(a*3-2, a*3+1):
                for k in range(b*3-2, b*3+1):
                    if((j,k) not in copy_state.prefills):
                        copy_state.state[(j,k)] = np.random.choice(copy_state.allowed_entries[(j,k)])
                
        return copy_state
    
    # A mutation function I tried out which turned out not to work.
    # Idea was to remove conflicts in high-fitness boards,
    # but for local maxima the conflics are circular. 
    def test_mutate(self):
        base = {1,2,3,4,5,6,7,8,9}
        for i in range(1, 10):
            row = self.get_row_state(i)
            j = [idx for idx, val in enumerate(row) if val in row[:idx] or val in row[idx+1:]] # search for duplicates in the row
            missing =  base - set(row)

            # change duplicate cell to missing number in row
            if(len(j) != 0):
                print("row:", i)
                print("index:", j)
                print("missing:", missing)
                print("-------------")
                if((i,j[0]) not in self.prefills): # len(j) should be 2 at this point
                    self.state[(i,j[0]+1)] = list(missing)[0]
                else:
                    self.state[(i,j[1]+1)] = list(missing)[0]
                    
        for j in range(1, 10):
            col = self.get_col_state(j)
            i = [idx for idx, val in enumerate(col) if val in col[:idx] or val in col[idx+1:]] # search for duplicates in the row
            missing =  base - set(col)

            # change duplicate cell to missing number in row
            if(len(i) != 0):
                print("col:", j)
                print("index:", i)
                print("missing:", missing)
                print("-------------")
                if((i[0],j) not in self.prefills): # len(j) should be 2 at this point
                    self.state[(i[0]+1,j)] = list(missing)[0]
                else:
                    self.state[(i[1]+1,j)] = list(missing)[0]

In [2]:
def read_sudoku_problem(filename, compute_allowed_fills=True, verbose=False):
    prefills = {}
    with open(filename, 'r') as file:
        row_id = 1
        for rows in file:
            rows = rows.strip()
            cont_list = [char for char in rows]
            for (col_id, row_contents) in enumerate(cont_list):
                row_contents = row_contents.strip()
                if '1' <= row_contents <= '9':
                    prefills[(row_id, col_id+1)] = int(row_contents)
            row_id = row_id + 1
        file.close()
    return SudokuBoard(prefills, compute_allowed_fills, verbose)

In [3]:
# swaps three random rows
def reproduce_v2(parent1, parent2):
    # pick 3 random rows to exchange
    rand_rows = sample(range(1,9), 3)
    rand_cols = sample(range(1,9), 3)
    rand_blks = [np.random.choice([1,2,3], size=2) for i in range(2)]
    
    p1_rows = [parent1.get_row_state(row) for row in rand_rows]
    p2_rows = [parent2.get_row_state(row) for row in rand_rows]
    p1_cols = [parent1.get_col_state(col) for col in rand_cols]
    p2_cols = [parent2.get_col_state(col) for col in rand_cols]
    p1_blks = [parent1.get_blk_state_by_blk_idx(rand_blks[i][0], rand_blks[i][1]) for i in range(2)]
    p2_blks = [parent2.get_blk_state_by_blk_idx(rand_blks[i][0], rand_blks[i][1]) for i in range(2)]
    
    child1 = parent1.create_child_v2(rand_rows, rand_cols, rand_blks, p2_cols, p2_rows, p2_blks)
    child2 = parent2.create_child_v2(rand_rows, rand_cols, rand_blks, p1_cols, p1_rows, p1_blks)

    return [child1, child2]


# input: list of randomly initialized states based on a single problem
# returns: a solved problem (hopefully)
def genetic_algorithm_v2(population, filename): 
    reproductions = int(len(population)/2)
    generation = 0
    best_fitness = []
    curr_best_fitness = -1000
    reset_counter = 0
    
    while(generation < 40):
        print("Generation:", generation)
        has_improved = False
        new_population = [] # holds children
        
        # use softmax function to compute probabilities
        fit_arr = [board.compute_fitness_score() for board in population] # fitness of current population for weighting random choice
        if(0 in fit_arr): return population[fit_arr.index(0)] # check for solution in population
        
        std_dev = np.std(fit_arr)
        average = np.average(fit_arr)
        fit_arr_std = [(x)/(std_dev) for x in fit_arr] # standardized fitness array
        sum_std_exp = sum(np.exp(fit_arr_std)) # sum of e^(standardized elements of fit_arr)
        
        p = [np.exp(score)/sum_std_exp for score in fit_arr_std] # map fitness scores to probabilities which sum to one

        # call the reproduction function on randomly chosen parents
        for i in range(reproductions):
            # note: mutation function is in new child creator: create_child_v2()
            parents = np.random.choice(population, size=2, p=p)
            children = reproduce_v2(parents[0], parents[1]) 
            
            new_population = new_population + children
            
            
            if(max(fit_arr) > curr_best_fitness):
                curr_best_fitness = max(fit_arr)
                best_fitness.append(curr_best_fitness)
                print("--> best fitness:", curr_best_fitness)
                reset_counter = 0
                has_improved = True
            
        population = new_population
        generation += 1
        
        # soft reset after 5 generations of no improvement
        if(has_improved == False):
            reset_counter += 1
        
        if(reset_counter > 5):
            print("-------soft reset-------")
            best_boards = sorted(population, key = lambda board: board.compute_fitness_score())[500:]
            population = best_boards + [read_sudoku_problem(filename) for i in range(1500)]
            best_boards[-1].pretty_print() # print the best board at the soft reset
            reset_counter = 0
    
    return best_fitness

In [5]:
pop = [read_sudoku_problem('ai_escargot.txt') for i in range(2000)]
x = genetic_algorithm_v2(pop, 'ai_escargot.txt')

Generation: 0
--> best fitness: -38
Generation: 1
Generation: 2
--> best fitness: -35
Generation: 3
--> best fitness: -30
Generation: 4
Generation: 5
Generation: 6
--> best fitness: -27
Generation: 7
--> best fitness: -26
Generation: 8
--> best fitness: -25
Generation: 9
Generation: 10
--> best fitness: -22
Generation: 11
--> best fitness: -17
Generation: 12
Generation: 13
--> best fitness: -16
Generation: 14
--> best fitness: -14
Generation: 15
--> best fitness: -12
Generation: 16
--> best fitness: -9
Generation: 17
--> best fitness: -7
Generation: 18
Generation: 19
Generation: 20
Generation: 21
Generation: 22
--> best fitness: -6
Generation: 23
Generation: 24
Generation: 25
Generation: 26
Generation: 27
Generation: 28
-------soft reset-------
|---------+---------+---------|
| 1  5  6 | 8  3  7 | 2  9  4 |
| 7  3  4 | 9  2  5 | 1  6  8 |
| 2  8  9 | 6  4  1 | 5  7  3 |
|---------+---------+---------|
| 8  7  5 | 3  6  2 | 9  4  1 |
| 4  1  3 | 7  8  9 | 6  5  2 |
| 6  9  2 | 5  1  4 |

# Version 1 Notes: 

**Reproduction function:** Selects three random rows and swaps them in the parents to produce the children.

**Muation:** None.

**Population:** 50 boards.

**Issues:** Usually converges on a fitness that isn't very good very quickly. My guess is that this is because the softmax function weights the board with the best fitness so much higher than the others that by the second generation the population is fairly homogeneous. 

**Fix:** The softmax function is very sensitive to variance in the sample, so I need to standardize the sample by dividing by some multiple of the standard deviation. An example of this is shown below.

In [88]:
# dry application of softmax
a = [1,2,3,4,5]
p_dist = [np.exp(ai)/sum(np.exp(a)) for ai in a]
print("Original sample:", a)
print("Original probability distribution:", p_dist)

Original sample: [1, 2, 3, 4, 5]
Original probability distribution: [0.011656230956039607, 0.03168492079612427, 0.0861285444362687, 0.23412165725273662, 0.6364086465588308]


In [90]:
# standardizing the original sample by dividing by 10 times the standard deviation
a2 = [(ai)/(10*np.std(a)) for ai in a]
p_dist2 = [np.exp(a2i)/sum(np.exp(a2)) for a2i in a2]
print("Standardized sample:", a2)
print("Standardized distribution:", p_dist2)

Standardized sample: [0.07071067811865475, 0.1414213562373095, 0.21213203435596426, 0.282842712474619, 0.35355339059327373]
Standardized distribution: [0.17275966625325476, 0.1854178810654429, 0.19900357163459742, 0.21358469472180378, 0.22923418632490114]


Notice how the second probability distribution gives the lower values a greater chance of being selected.

# Version 2 Observations: 

**Reproduction function:** Swaps three random rows, then three random columns, then two random blocks.

**Muation:** Picks a random block and re-initializes all the non-prefill values to a ranom value in the allowed values.

**Population:** 2000 boards.

**Comments:** Sriram recommended I increase the population by a lot. When the population was 1000, the population would still converge, albiet much more slowly. 2000 boards seems to be where the returns diminish on population size. At this size or greater it wouldn't converge to a single configuration anymore. Larger sizes beyond this didn't produce an improvement in performance. In this version, the program solves easy boards in a single generation, but consistently peaks at a fitness of around -4 for medium and hard puzzles. The types of local maxima the board gets stuck in are very particular: Though they only have a few conflicts, resolving one conflict creates another ad infinitum. I added a feature where if no improvement was seen over some number of generations, then the next generation would consist of some of the highest-fitness boards from that generation, with the rest being a new set of random boards. This did not improve performance in the amount of time I was letting the program run. 

## Summary of Development:

The first version used a very straightforward an naive strategy. There was no mutation function, the crossover function swapped three random rows between parents to produce children, and the population was only 50. The population converged *on a single board configuration* within 5 or 6 generations on average. The problem here was twofold: First, the softmax function, which I used to map the fitness of a generation to a selection probability, is really sensitive to variance. So if the range of fitness scores in the initial population was 15, for example, the board with the highest score would have a pobability of being chosen of around 0.9 *even if the next highest board's fitness was only one point lower.* What I did to fix this was increase the population to a value in the thousands, and divide the fitness of the boards by the standard deviation of the population before applying the softmax function. This alone saw the average best fitness after running the algorithm half from around -40 to around -20. When playing around with the population size in this iteration and the final one, I noticed that increasing it beyond 2000 boards didn't show better results in terms of fitness. The next thing to do at this stage was write a crossover function that swapped columns and blocks in addition to rows, and also add a mutation function. Doing this brought the average fitness up to more than -10 from a previous best of -20 on average. At this point I figured that if there are only a handful of conflicts left I could resolve them manually by searching for them and changing their value to the one missing from that row or column. This is the function `test_mutate`. However, I realized by studying some of the high-fitness boards that the algorithm would produce that resolving a conflict would *always* create another one, so the configuration was truly a local max. I added a feature where if the global best fitness score didn't improve over five generations then I would keep the best 500 boards and replace the rest with randomly initialized ones. This strategy didn't have any noticable impact over a 40-generation run, since the fitness would usually bottom out around gereration 30 and I only have so much patience.    

## Conclusion:

Backtracking is a more effective way to solve Sukoku boards of this size than genetic algorithms or simulated annealing. For tougher puzzles, there is too much luck required to solve boards by these advanced guess-and-check methods, given there seems to be a large number of local maxima very close to a fitness of zero. The main insight I took away from doing this project is that the art of implementing genetic algorithms and simulated annealing is in getting them to converge at the right rate. This means keeping the population sufficiently diverse while having a system which guarantees improvement. Another interesting observation is that GA, unlike simulated annealing, appears to increase fitness linearly right up until it converges on a local maximum. There is no phase change period where most of the improvement is seen. 